In [2]:
from llama_index.llms.gemini import Gemini
from dotenv import load_dotenv
load_dotenv()
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.core import Settings

Settings.llm = Gemini(model="models/gemini-1.5-flash-latest", temperature=0)

def check_hotel_reservation_availability(date:str) -> str:
    """Check if hotel reservations are available for the given date"""
    if date == "Tomorrow" or date == "tomorrow":
        return "Yes, Reservations are available for Tomorrow"
    else:
        return "Sorry, Reservations are only available for Tomorrow at the moment"

check_hotel_reservation_availability_tool = FunctionTool.from_defaults(fn=check_hotel_reservation_availability)

In [3]:
from llama_index.embeddings.gemini import GeminiEmbedding

gemini_embedding_model = GeminiEmbedding(model_name="models/embedding-001")
Settings.embed_model = gemini_embedding_model

In [6]:
from llama_index.core import VectorStoreIndex, Settings
from llama_parse import LlamaParse
import nest_asyncio
nest_asyncio.apply()
documents = LlamaParse(result_type="markdown").load_data("./data/knowledge.pdf")

Started parsing the file under job_id f792e11e-bb3c-4620-b29f-6bc9fd104c27


In [8]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# create your index
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)
query_engine = index.as_query_engine()

d:\learning\AI\demo-doubts\agents\agent\my_env\Lib\site-packages\tiktoken\core.py:50: RuntimeWarning: coroutine 'LlamaParse.aload_data' was never awaited
  self._core_bpe = _tiktoken.CoreBPE(mergeable_ranks, special_tokens, pat_str)


In [9]:
from llama_index.core.tools import QueryEngineTool
hotel_policy_and_services_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="Hotel",
    description="Get the policy and services related information for a hotel",
)

In [10]:
agent = ReActAgent.from_tools([check_hotel_reservation_availability_tool, hotel_policy_and_services_tool], llm=Settings.llm, verbose=True)

In [12]:
response = agent.chat(
    "I have to book a hotel reservation for tommorow please let me know if it is available and also provide me with the pricing details",
)

print(response)

> Running step 21a9e663-a690-4db5-995c-4df2db29362d. Step input: I have to book a hotel reservation for tommorow please let me know if it is available and also provide me with the pricing details
Thought: I need to check if the hotel has reservations available for tomorrow.
Action: check_hotel_reservation_availability
Action Input: {'date': 'tomorrow'}
Observation: Yes, Reservations are available for Tomorrow
> Running step fa488088-a62b-4019-a7aa-9b57c451e51b. Step input: None
Thought: I need to get the pricing details for the hotel.
Action: Hotel
Action Input: {'input': 'pricing details'}
Observation: The hotel offers three room types: Standard, Deluxe, and Executive Suite. The Standard Room costs 7,000 per night, the Deluxe Room costs 10,500 per night, and the Executive Suite costs 17,500 per night. 

> Running step 6f70f83e-08a4-4867-90f2-d668a902a172. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Yes, reservatio

In [5]:
from llama_index.core import SummaryIndex
from llama_index.readers.web import FireCrawlWebReader
from IPython.display import Markdown, display
import os



[Document(id_='338bf4ea-2808-4e24-aad8-f71403a79539', embedding=None, metadata={'title': 'Gujarat - Wikipedia', 'robots': 'max-image-preview:standard', 'ogTitle': 'Gujarat - Wikipedia', 'ogImage': 'https://upload.wikimedia.org/wikipedia/commons/thumb/2/21/GujaratPhotoMontage.jpg/1200px-GujaratPhotoMontage.jpg', 'ogLocaleAlternate': [], 'sourceURL': 'https://en.wikipedia.org/wiki/Gujarat', 'pageStatusCode': 200}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[Jump to content](#bodyContent)\n\n Main menu\n\nMain menu\n\nmove to sidebar hide\n\nNavigation\n\n*   [Main page](/wiki/Main_Page "Visit the main page [z]")\n    \n*   [Contents](/wiki/Wikipedia:Contents "Guides to browsing Wikipedia")\n    \n*   [Current events](/wiki/Portal:Current_events "Articles related to current events")\n    \n*   [Random article](/wiki/Special:Random "Visit a randomly selected article [x]")\n    \n*   [About Wikipedia](/wiki/Wikipedia:About "Learn about Wikipedia 

In [2]:
l1 = [1,2,3,4]
l2 = [5,6,7,8]

l1.extend(l2)
l1

[1, 2, 3, 4, 5, 6, 7, 8]

In [12]:
documents

[Document(id_='338bf4ea-2808-4e24-aad8-f71403a79539', embedding=None, metadata={'title': 'Gujarat - Wikipedia', 'robots': 'max-image-preview:standard', 'ogTitle': 'Gujarat - Wikipedia', 'ogImage': 'https://upload.wikimedia.org/wikipedia/commons/thumb/2/21/GujaratPhotoMontage.jpg/1200px-GujaratPhotoMontage.jpg', 'ogLocaleAlternate': [], 'sourceURL': 'https://en.wikipedia.org/wiki/Gujarat', 'pageStatusCode': 200}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[Jump to content](#bodyContent)\n\n Main menu\n\nMain menu\n\nmove to sidebar hide\n\nNavigation\n\n*   [Main page](/wiki/Main_Page "Visit the main page [z]")\n    \n*   [Contents](/wiki/Wikipedia:Contents "Guides to browsing Wikipedia")\n    \n*   [Current events](/wiki/Portal:Current_events "Articles related to current events")\n    \n*   [Random article](/wiki/Special:Random "Visit a randomly selected article [x]")\n    \n*   [About Wikipedia](/wiki/Wikipedia:About "Learn about Wikipedia 

In [3]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext, VectorStoreIndex


def initialize_vector_store(collection_name: str, documents = None):
    """Configure and initialize the ChromaDB for vector storage."""
    db = chromadb.PersistentClient(path="./src/chroma_db")
    chroma_collection = db.get_or_create_collection(collection_name)
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
  # Conditionally load documents into the vector store
    if documents is None:
        index = VectorStoreIndex.from_vector_store(vector_store=vector_store, storage_context=storage_context)
    else:
        index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

    return index.as_query_engine()

In [7]:
from llama_index.readers.web import FireCrawlWebReader
from llama_parse import LlamaParse
import nest_asyncio
def load_local_data():
    nest_asyncio.apply()
    return LlamaParse(result_type="markdown").load_data("./src/data/knowledge.pdf")

In [14]:
db = chromadb.PersistentClient(path="./src/chroma_db")
# check if the collection exists
collections = db.list_collections()

print(collections)

for item in collections:
    if item.name == "hotel_knowledge":
        print("Collection exists")

[Collection(id=6aaa988e-b0a7-4f06-a10c-541bcd890ed7, name=hotel_knowledge)]
Collection exists


In [8]:
def hotel_query_engine():
    db = chromadb.PersistentClient(path="./src/chroma_db")
    # check if the collection exists
    collections = db.list_collections()
    if "hotel_knowledge" in collections:
        return initialize_vector_store("hotel_knowledge")
    else:
        hotel_data = load_local_data()
        return initialize_vector_store("hotel_knowledge", hotel_data)

hotel_query_engine()

Started parsing the file under job_id cac11eca-275f-4b0f-bb92-eeb96142a25f


ValueError: 
******
Could not load OpenAI embedding model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

Consider using embed_model='local'.
Visit our documentation for more embedding options: https://docs.llamaindex.ai/en/stable/module_guides/models/embeddings.html#modules
******

In [2]:
from llama_index.readers.web import FireCrawlWebReader

def load_web_data(url: str):
    firecrawl_reader = FireCrawlWebReader(
        api_key="fc-5a058f253d75495eba0a49fc9c07b2e6",
        mode="scrape",
        params={"additional": "parameters"},
    )
    return firecrawl_reader.load_data(url=url)

a = load_web_data("https://www.google.com")
a

KeyboardInterrupt: 